In [1]:
import matplotlib.pyplot as plt # for plotting
import numpy as np # for transformation
import pandas as pd

import torch # PyTorch package
import torchvision # load datasets
import torchvision.transforms as transforms # transform data
import torch.nn as nn # basic building block for neural neteorks
import torch.nn.functional as F # import convolution functions like Relu
import torch.optim as optim # optimzer

ModuleNotFoundError: No module named 'torch'

In [ ]:
# Load the JSON data into a python dictionary
train_data = pd.read_json("train_data.json")

# Function to expand the dictionaries into separate rows
def expand_dict(row):
    return pd.json_normalize(row)

# Apply the function to each row and concatenate the results
expanded_data = pd.concat(train_data.apply(expand_dict, axis=1).tolist(), ignore_index=True)

# Print the expanded DataFrame
print(expanded_data)

In [ ]:
#Exploration and minor cleaning
print(expanded_data.columns)

print(expanded_data["sentiment"].value_counts())

dupe = expanded_data["appid"].value_counts()
if 2 in dupe.values:
    print("There is at least one value with a count of 2.")
else:
    print("No value has a count of 2.")

# Clean out the games that have no reviews
train_df = expanded_data.dropna(subset=["sentiment"])

In [ ]:
# python image library of range [0, 1] 
# transform them to tensors of normalized range[-1, 1]

transform = transforms.Compose( # composing several transforms together
    [transforms.ToTensor(), # to tensor object
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]) # mean = 0.5, std = 0.5

# set batch_size
batch_size = 4

# set number of workers
num_workers = 2

# load train data
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=num_workers)

# load test data
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=num_workers)

# put 10 classes into a set
classes = ('Overwhelmingly Positive', 'Very Positive', 'Positive', 'Mostly Positive',
           'Mixed', 'Mostly Negative', 'Negative', 'Very Negative', 'Ovewhelmingly Negative')